In [1]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.widgets import RunDetails

from azureml.core import Dataset

from azureml.pipeline.core import Pipeline, PipelineData, PipelineRun, StepRun, PortDataReference
from azureml.pipeline.steps import PythonScriptStep

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

from azureml.core.model import Model

import warnings
warnings.filterwarnings('ignore')

print('SDK Version:', azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cloudpickle 2.0.0 (/opt/anaconda3/lib/python3.8/site-packages), Requirement.parse('cloudpickle<2.0.0,>=1.1.0'), {'azureml-dataprep'}).


SDK Version: 1.35.0


In [2]:
# load workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: hw2_pipeline
Azure region: centralus
Subscription id: 441cf665-f2bb-42da-9467-9dc54a3ae431
Resource group: pipeline


In [3]:
# Get the default datastore
default_ds = ws.get_default_datastore()

# Enumerate all datastores, indicating which is the default
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

azureml_globaldatasets - Default = False
workspaceblobstore - Default = True
workspacefilestore - Default = False
workspaceworkingdirectory - Default = False
workspaceartifactstore - Default = False


In [4]:
if 'churn dataset' not in ws.datasets:
    default_ds.upload_files(files=['/Users/lingyizhao/Desktop/new_data.csv'], # Upload the diabetes csv files in /data
                        target_path='churn-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'churn-data/new_data.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='churn dataset',
                                description='churn data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


In [5]:
import os
# Create a folder for the pipeline step files
experiment_folder = 'churn_pipeline'
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

churn_pipeline


In [6]:
%%writefile $experiment_folder/prep_churn.py
# Import libraries
import os
import argparse
import pandas as pd
from azureml.core import Run
from sklearn.preprocessing import MinMaxScaler

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str, dest='raw_dataset_id', help='raw dataset')
parser.add_argument('--prepped-data', type=str, dest='prepped_data', default='prepped_data', help='Folder for results')
args = parser.parse_args()
save_folder = args.prepped_data

# Get the experiment run context
run = Run.get_context()

# load the data (passed as an input dataset)
print("Loading Data...")
churn = run.input_datasets['raw_data'].to_pandas_dataframe()

# Log raw row count
row_count = (len(churn))
run.log('raw_rows', row_count)

# data cleaning
churn = churn.dropna()

# Log processed rows
row_count = (len(churn))
run.log('processed_rows', row_count)

# Save the prepped data
print("Saving Data...")
os.makedirs(save_folder, exist_ok=True)
save_path = os.path.join(save_folder,'data.csv')
churn.to_csv(save_path, index=False, header=True)

# End the run
run.complete()

Writing churn_pipeline/prep_churn.py


In [7]:
%%writefile $experiment_folder/train_churn.py
# Import libraries
from azureml.core import Run, Model
import argparse
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument("--training-folder", type=str, dest='training_folder', help='training data folder')
args = parser.parse_args()
training_folder = args.training_folder

# Get the experiment run context
run = Run.get_context()

# load the prepared data file in the training folder
print("Loading Data...")
file_path = os.path.join(training_folder,'data.csv')
churn = pd.read_csv(file_path)

X = churn[churn.columns.difference(['Attrition_Yes'])].values
y = churn['Attrition_Yes'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# Scale the data, using standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Train the random forest model
print('Training a random forest model...')
model = RandomForestClassifier().fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# plot ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
fig = plt.figure(figsize=(6, 4))
# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], 'k--')
# Plot the FPR and TPR achieved by our model
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
run.log_image(name = "ROC", plot = fig)
plt.show()

# Save the trained model in the outputs folder
print("Saving model...")
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'randomforest_model.pkl')
joblib.dump(value=model, filename=model_file)

# Register the model
print('Registering model...')
Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'randomforest_model',
               tags={'Training context':'Pipeline'},
               properties={'AUC': np.float(auc), 'Accuracy': np.float(acc)})


run.complete()

Writing churn_pipeline/train_churn.py


In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "mlop-test"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [9]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration

# Create a Python environment for the experiment
churn_env = Environment("churn-pipeline-env")

# Create a set of package dependencies
churn_packages = CondaDependencies.create(conda_packages=['scikit-learn','ipykernel','matplotlib','pandas','pip'],
                                             pip_packages=['azureml-defaults','azureml-dataprep[pandas]','pyarrow'])

# Add the dependencies to the environment
churn_env.python.conda_dependencies = churn_packages

# Register the environment 
churn_env.register(workspace=ws)
registered_env = Environment.get(ws, 'churn-pipeline-env')

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above. 
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


In [10]:
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
churn_ds = ws.datasets.get("churn dataset")

# Create a PipelineData (temporary Data Reference) for the model folder
prepped_data_folder = PipelineData("prepped_data_folder", datastore=ws.get_default_datastore())

# Step 1, Run the data prep script
train_step = PythonScriptStep(name = "Prepare Data",
                                source_directory = experiment_folder,
                                script_name = "prep_churn.py",
                                arguments = ['--input-data', churn_ds.as_named_input('raw_data'),
                                             '--prepped-data', prepped_data_folder],
                                outputs=[prepped_data_folder],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
register_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "train_churn.py",
                                arguments = ['--training-folder', prepped_data_folder],
                                inputs=[prepped_data_folder],
                                compute_target = pipeline_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

print("Pipeline steps defined")

Pipeline steps defined


In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

# Construct the pipeline
pipeline_steps = [train_step, register_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)
print("Pipeline is built.")

# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'churn-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

Pipeline is built.
Created step Prepare Data [8702eff8][04cfee9f-3773-49e2-9e65-179237aa0ca5], (This step will run and generate new outputs)Created step Train and Register Model [5b262813][a9bc6a29-edfc-41b6-9b6d-20d59340668c], (This step will run and generate new outputs)

Submitted PipelineRun 9bc15bf9-734b-4c2f-9049-de538d8d444f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/9bc15bf9-734b-4c2f-9049-de538d8d444f?wsid=/subscriptions/441cf665-f2bb-42da-9467-9dc54a3ae431/resourcegroups/pipeline/workspaces/hw2_pipeline&tid=6d7adc86-8446-4cb9-9972-1477c831ff28
Pipeline submitted for execution.


_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: 9bc15bf9-734b-4c2f-9049-de538d8d444f
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/9bc15bf9-734b-4c2f-9049-de538d8d444f?wsid=/subscriptions/441cf665-f2bb-42da-9467-9dc54a3ae431/resourcegroups/pipeline/workspaces/hw2_pipeline&tid=6d7adc86-8446-4cb9-9972-1477c831ff28
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: af214cf5-8bda-4418-8cc8-d1dcf4813860
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/af214cf5-8bda-4418-8cc8-d1dcf4813860?wsid=/subscriptions/441cf665-f2bb-42da-9467-9dc54a3ae431/resourcegroups/pipeline/workspaces/hw2_pipeline&tid=6d7adc86-8446-4cb9-9972-1477c831ff28
StepRun( Prepare Data ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_53bb39c91e5339d39dd46dc46ed72174a017c78327fbfa2f7f38d3d35f9977ea_d.txt
2021-11-04T20:03:32Z Running following command: /bin/bash -c sudo blobfuse /mnt/batch/tasks/shared/LS_root/jobs/hw2_pipeline/azureml/af214cf5-8bda-4418-8cc8-d1dcf4813860/mounts/work


Streaming azureml-logs/75_job_post-tvmps_53bb39c91e5339d39dd46dc46ed72174a017c78327fbfa2f7f38d3d35f9977ea_d.txt
[2021-11-04T20:03:50.873909] Entering job release
[2021-11-04T20:03:51.640429] Starting job release
[2021-11-04T20:03:51.641503] Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 313
[2021-11-04T20:03:51.641892] job release stage : upload_datastore starting...
[2021-11-04T20:03:51.648920] job release stage : start importing azureml.history._tracking in run_history_release.
[2021-11-04T20:03:51.648949] job release stage : execute_job_release starting...
[2021-11-04T20:03:51.649717] job release stage : copy_batchai_cached_logs starting...
[2021-11-04T20:03:51.649878] job release stage : copy_batchai_cached_logs completed...
[2021-11-04T20:03:51.650475] Entering context manager injector.
[2021-11-04T20:03:51.664761] job release stage : upload_datastore completed...
[2021-11-04T20:03:51.7765